# Modeling Configuration Notebook
Make sure you have already gone through the setup_guide.ipynb notebook. If you are using just a singular runtime on Google Colab, simply extend these cells to the setup_guide.ipynb for a continuous workflow.

Citing: https://medium.com/stanford-cs224w/scene-graph-generation-compression-and-classification-on-action-genome-dataset-9f692a1d5394

## Setting Directories

In [ ]:
!mkdir glove
!mkdir checkpoints
!mkdir checkpoints/custom_images
!mkdir checkpoints/your_output_path

## Define Path Variables

In [ ]:
# Path to glove directory
glove_dir = '/glove' 

# Path to the pretrained model
pretrained_model_path = '/upload_causal_motif_sgdet/'

# Path to output directory for default VGG
output_dir = '/upload_causal_motif_sgdet/' 

# If evaluation should be done on custom images, set this 'True'
test_custom_images = True

# The path for custom images for SGG detection, if TEST.CUSTUM_EVAL is 'True'
images_path = '/imgs'

# The path for output of SGG for custom images, if TEST.CUSTUM_EVAL is 'True'
sgg_output = '/results'

# Path to configuration file for Unbiased TDE
configuration_file = "sgg-relevance-scoring/scene-graph-benchmark/configs/e2e_relation_X_101_32_8_FPN_1x.yaml" 

## Modeling

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python -m torch.distributed.launch \
--master_port 10027 --nproc_per_node=1 tools/relation_test_net.py \
--config-file f"{configuration_file}" \
MODEL.ROI_RELATION_HEAD.USE_GT_BOX False \
MODEL.ROI_RELATION_HEAD.USE_GT_OBJECT_LABEL False \
MODEL.ROI_RELATION_HEAD.PREDICTOR CausalAnalysisPredictor \
MODEL.ROI_RELATION_HEAD.CAUSAL.EFFECT_TYPE TDE \
MODEL.ROI_RELATION_HEAD.CAUSAL.FUSION_TYPE sum \
MODEL.ROI_RELATION_HEAD.CAUSAL.CONTEXT_LAYER motifs \
TEST.IMS_PER_BATCH 1 \
DTYPE "float16" \
GLOVE_DIR "{glove_dir}" \
MODEL.PRETRAINED_DETECTOR_CKPT "{pretrained_model_path}" \
OUTPUT_DIR "{output_dir}" \
TEST.CUSTUM_EVAL {test_custom_images} \
TEST.CUSTUM_PATH "{images_path}" \
DETECTED_SGG_DIR "{sgg_output}"